# Parameter Sweep for Algorithmic Trading Analyser that uses a combination of two Trading Indicators: 
## Stochastic Oscillator and Simple Moving Averages

In [1]:
# Initial imports
import os
import requests
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt

%matplotlib inline

## Read Daily Market Closing Prices using the file generated by Data_generator.ipynb Notebook

In [2]:
faangm_closing_return_df_=pd.read_csv(Path("AMZN.csv"))
faangm_closing_return_df_.index = faangm_closing_return_df_["Unnamed: 0"]
faangm_closing_return_df_.drop(columns="Unnamed: 0", inplace=True)
faangm_closing_return_df_.head()

,AMZN
Unnamed: 0,
2017-09-26,938.59
2017-09-27,950.88
2017-09-28,956.40
2017-09-29,960.49
2017-10-02,958.79


## The follwing code runs the Parameter sweep for this model
### The output is store into "param_sweep.csv"

In [ ]:
# Initialze Lists to be used for creation of Datafreame having Input Parameters, and output Algo Trading Backtesting results
# Lists for Input Params
k_period_list=[]
d_period_list=[]
short_window_list=[]
long_window_list=[]
k_80_list=[]
k_20_list=[]

# Lists for Backtesting results
annual_return_list=[]
cumulative_returns_list=[]
annual_volatility_list=[]
sharpe_ratio_list=[]
sortino_ratio_list=[]


# 'For' loops that use each intended combination of Parameters to run the full Algo Trading Model and Backtesting 
# and append the results in the Lists initialized above
for k_period in range(12,21): # number of periods to calculate the %K line of the Stochastic Oscillator 
    for d_period in range(3,8): # number of periods to calculate the %D line of the Stochastic Oscillator 
        for short_window in range(5,11): # Short Window to calculate the Simple Moving Average Fast Indicator
            for long_window in range(10,16): # Long Window to calculate the Simple Moving Average Slow Indicator
                for k_80 in range(80,95): # Parameter to configure the overbought line of the Stochastic Oscillator 
                    for k_20 in range(20,29): # Parameter to configure the oversold line of the Stochastic Oscillator 
                        
                        # For each iteration of the innermost loop, create a copy of the dataframe having Market Closing prices
                        # This dataframe woudl be used to add columns per the alog trading workflow for this iteration
                        faangm_closing_return_df= faangm_closing_return_df_.copy()
                        
                        # Create Stochastic Oscillator Trading Indicators
                        # % K Line
                        # k_period=k_period
                        faangm_closing_return_df["AMZN_k_period_min"] = faangm_closing_return_df["AMZN"].rolling(window=k_period).min()
                        faangm_closing_return_df["AMZN_k_period_max"] = faangm_closing_return_df["AMZN"].rolling(window=k_period).max()
                        faangm_closing_return_df["AMZN_k_component_fast"] =  100 * ((faangm_closing_return_df["AMZN"] - faangm_closing_return_df["AMZN_k_period_min"]) / (faangm_closing_return_df["AMZN_k_period_max"] - faangm_closing_return_df["AMZN_k_period_min"]))
                        
                        # %D Line
                        # d_period=d_period
                        faangm_closing_return_df["AMZN_d_component_slow"] = faangm_closing_return_df["AMZN_k_component_fast"].rolling(window=d_period).mean()
                        
                        # Simple Movign Average Trading Indicators
                        # short_window = short_window
                        faangm_closing_return_df["AMZN_sma_fast"] = faangm_closing_return_df["AMZN"] .rolling(window=short_window).mean()
                        
                        # long_window = long_window
                        faangm_closing_return_df["AMZN_sma_slow"] = faangm_closing_return_df["AMZN"] .rolling(window=long_window).mean()
                        
                        # Drop NaN records which were created for the window at the begining of the rolling windows
                        faangm_closing_return_df = faangm_closing_return_df.dropna()
                        
                        # The following code uses the Trading Indicators to create Trade Postitions and Signals
                        # Define Lists to store the Trading Indicators
                        prices= faangm_closing_return_df["AMZN"].array
                        k= faangm_closing_return_df["AMZN_k_component_fast"].array
                        d= faangm_closing_return_df["AMZN_d_component_slow"].array
                        sma_fast= faangm_closing_return_df["AMZN_sma_fast"].array
                        sma_slow= faangm_closing_return_df["AMZN_sma_slow"].array
                        
                        # Initialize lists to store the Trading Signals
                        buy_price = []
                        sell_price = []
                        stoch_signal = []
                        signal = 0
                        
                        
                        # For loop that uses the Market Closing prices and Trading Indicators to create Signals for Buy and Sell Tardes
                        for i in range(len(prices)):
                            if (k[i] < k_20 and d[i] < k_20 and k[i] < d[i])  or ((sma_fast[i] > sma_slow[i]) and (sma_fast[i-1] <= sma_slow[i-1])) :
                                if signal != 1:
                                    buy_price.append(prices[i])
                                    sell_price.append(np.nan)
                                    signal = 1                
                                    stoch_signal.append(signal)
                                else:
                                    buy_price.append(np.nan)
                                    sell_price.append(np.nan)
                                    stoch_signal.append(0)
                            elif (k[i] > k_80 and d[i] > k_80 and k[i] > d[i]) or ((sma_fast[i] < sma_slow[i]) and (sma_fast[i-1] >= sma_slow[i-1])):
                                if signal != -1:
                                    buy_price.append(np.nan)
                                    sell_price.append(prices[i])
                                    signal = -1                
                                    stoch_signal.append(signal)
                                else:
                                    buy_price.append(np.nan)
                                    sell_price.append(np.nan)
                                    stoch_signal.append(0)
                            else:
                                buy_price.append(np.nan)
                                sell_price.append(np.nan)
                                stoch_signal.append(0)                             
                                


                        faangm_closing_return_df["AMZN_stoch_oscl_exit"]= stoch_signal
                        faangm_closing_return_df["AMZN_stoch_oscl_signal"]= faangm_closing_return_df["AMZN_stoch_oscl_exit"].cumsum()
                        # faangm_closing_return_df["buy_price"] = buy_price
                        # faangm_closing_return_df["sell_price"] = sell_price
                        faangm_closing_return_df["%K_80"]= k_80
                        faangm_closing_return_df["%K_20"]= k_20
                        
                        
                        
                        
                        # Drop all NaN values from the DataFrame
                        faangm_closing_return_df = faangm_closing_return_df.dropna()

                        # Set initial capital
                        initial_capital = float(1100000)

                        # Set the share size
                        share_size = 500

                        # Take a 500 share position where the dual moving average crossover is 1 (sma_fast is greater than sma_slow)
                        faangm_closing_return_df['AMZN_position'] = share_size * faangm_closing_return_df['AMZN_stoch_oscl_signal']

                        # Find the points in time where a 500 share position is bought or sold
                        faangm_closing_return_df['AMZN_entry_exit_position'] = faangm_closing_return_df['AMZN_position'].diff()

                        # Multiply share price by entry/exit positions and get the cumulatively sum
                        faangm_closing_return_df['AMZN_portfolio_holdings'] = faangm_closing_return_df['AMZN'] * faangm_closing_return_df['AMZN_entry_exit_position'].cumsum()

                        # Subtract the initial capital by the portfolio holdings to get the amount of liquid cash in the portfolio
                        faangm_closing_return_df['AMZN_portfolio_cash'] = initial_capital - (faangm_closing_return_df['AMZN'] * faangm_closing_return_df['AMZN_entry_exit_position']).cumsum()

                        # Get the total portfolio value by adding the cash amount by the portfolio holdings (or investments)
                        faangm_closing_return_df['AMZN_portfolio_total'] = faangm_closing_return_df['AMZN_portfolio_cash'] + faangm_closing_return_df['AMZN_portfolio_holdings']

                        # Calculate the portfolio daily returns
                        faangm_closing_return_df['AMZN_portfolio_daily_returns'] = faangm_closing_return_df['AMZN_portfolio_total'].pct_change()

                        # Calculate the cumulative returns
                        faangm_closing_return_df['AMZN_portfolio_cumulative_returns'] = (1 + faangm_closing_return_df['AMZN_portfolio_daily_returns']).cumprod() - 1

                        # Drop all NaN values from the DataFrame
                        faangm_closing_return_df = faangm_closing_return_df.dropna()
                        
                        
                        
                        
                        k_period_list.append(k_period)
                        d_period_list.append(d_period)
                        short_window_list.append(short_window)
                        long_window_list.append(long_window)
                        k_80_list.append(k_80)
                        k_20_list.append(k_20)
                        
                        # Calculate cumulative return
                        cumulative_returns_list.append(faangm_closing_return_df['AMZN_portfolio_cumulative_returns'][-1])

                        # Calculate annualized return
                        annual_return_list.append( (faangm_closing_return_df['AMZN_portfolio_daily_returns'].mean() * 252) )

                        # Calculate annual volatility
                        annual_volatility_list.append( ( faangm_closing_return_df['AMZN_portfolio_daily_returns'].std() * np.sqrt(252) ))

                        # Calculate Sharpe Ratio
                        sharpe_ratio_list.append( ( faangm_closing_return_df['AMZN_portfolio_daily_returns'].mean() * 252) / ( faangm_closing_return_df['AMZN_portfolio_daily_returns'].std() * np.sqrt(252)))

                        # Calculate Downside Return
                        sortino_ratio_df = faangm_closing_return_df[['AMZN_portfolio_daily_returns']].copy()
                        sortino_ratio_df.loc[:,'downside_returns'] = 0

                        target = 0
                        mask = sortino_ratio_df['AMZN_portfolio_daily_returns'] < target
                        sortino_ratio_df.loc[mask, 'downside_returns'] = sortino_ratio_df['AMZN_portfolio_daily_returns']**2

                        # Calculate Sortino Ratio
                        down_stdev = np.sqrt(sortino_ratio_df['downside_returns'].mean()) * np.sqrt(252) # Annualizing
                        expected_return = sortino_ratio_df['AMZN_portfolio_daily_returns'].mean() * 252 # Annualizing
                        sortino_ratio = expected_return/down_stdev

                        sortino_ratio_list.append(sortino_ratio)




                
    


# Prepare DataFrame for metrics
# metrics = ['k_period','d_period','short_window','long_window','k_80','k_20','annual_return', 'cumulative_returns','annual_volatility','sharpe_ratio','sortino_ratio']

# columns = ['backtest']

# Initialize the DataFrame with index set to evaluation metrics and column as `Backtest` (just like PyFolio)
#portfolio_evaluation_df = pd.DataFrame(index=metrics, columns=columns)
AMZN_portfolio_evaluation_df = pd.DataFrame({ 'k_period': k_period_list ,
                                             'd_period': d_period_list ,
                                             'short_window': short_window_list ,
                                             'long_window': long_window_list ,
                                             'k_80': k_80_list ,
                                             'k_20': k_20_list ,
                                             'annual_return': annual_return_list ,
                                             'cumulative_returns': cumulative_returns_list ,
                                             'annual_volatility': annual_volatility_list,
                                             'sharpe_ratio': sharpe_ratio_list,
                                             'sortino_ratio': sortino_ratio_list })


AMZN_portfolio_evaluation_df.to_csv("param_sweep.csv")

AMZN_portfolio_evaluation_df.head()